<h2>Establish connection</h2>

In [1]:
%reload_ext google.cloud.bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:\\Users\\Cermati\\AppData\\Roaming\\gcloud\\application_default_credentials.json"
print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])
from google.cloud import bigquery

import pandas as pd
import numpy as np
import Limit_util as c_lu
lu = c_lu.Limit_util()

C:\Users\Cermati\AppData\Roaming\gcloud\application_default_credentials.json


<h2>Get Data from BQ - Upgrade Limit group</h2>

In [2]:
grouped_cla = lu.get_group_cla()

# print(grouped_cla.head())
# print(grouped_cla.dtypes)

# check for duplicate orderId
print(grouped_cla.groupby(['updater_role','action','log_date','fee_scheme'],dropna=False).agg({
    'orderId' : 'count'
    # 'orderId' : ['count',pd.Series.nunique]
}))

C:\Users\Cermati\Anaconda3\lib\site-packages\google\auth\_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


                                                                               orderId
updater_role action                       log_date   fee_scheme                       
WORKER       UPGRADE_CREDIT_LIMIT_ACCOUNT 2021-07-23 BLIBLI_WHITELABEL_SCHEME    14572
data-analyst UPGRADE_CREDIT_LIMIT_ACCOUNT 2021-02-09 INDODANA_GENERIC_SCHEME      6782
                                          2021-04-14 INDODANA_GENERIC_SCHEME      3539
                                          2021-04-16 TIKET_WHITELABEL_SCHEME     13409
                                          2021-06-21 TIKET_WHITELABEL_SCHEME     10301
developers   UPGRADE_CREDIT_LIMIT_ACCOUNT 2021-06-23 INDODANA_GENERIC_SCHEME     34340
                                                     TIKET_WHITELABEL_SCHEME     34340


<h2>Get Data from BQ - User Historical records</h2>

In [3]:
# To compare to certain days after
# historical_data15 = lu.get_historical_data(-1,15)
# historical_data30 = lu.get_historical_data(-1,30)
# historical_data45 = lu.get_historical_data(-1,45)
# historical_data60 = lu.get_historical_data(-1,60)
# historical_data90 = lu.get_historical_data(-1,90)
# historical_data15_1 = lu.get_historical_data_1(-1,15)
# historical_data30_1 = lu.get_historical_data_1(-1,30)
# historical_data45_1 = lu.get_historical_data_1(-1,45)
# historical_data60_1 = lu.get_historical_data_1(-1,60)
# historical_data90_1 = lu.get_historical_data_1(-1,90)
# historical_data = pd.concat([historical_data15,historical_data30,historical_data45,historical_data60,historical_data90,
#                             historical_data15_1,historical_data30_1,historical_data45_1,historical_data60_1,historical_data90_1])

# To compare to yesterday data
historical_data_toCurrentDate = lu.get_historical_data_toCurrentDate(-1)
historical_data_1_toCurrentDate = lu.get_historical_data_1_toCurrentDate(-1)
historical_data = pd.concat([historical_data_toCurrentDate,historical_data_1_toCurrentDate])

historical_data['old_limit'] = historical_data['old_limit'].astype(float)
historical_data['new_limit'] = historical_data['new_limit'].astype(float)
historical_data['limit_usage_old'] = historical_data['limit_usage_old'].astype(float)
historical_data['limit_usage_new'] = historical_data['limit_usage_new'].astype(float)
historical_data['limit_addition_amount'] = historical_data['new_limit'] - historical_data['old_limit']
historical_data['limit_addition_rate'] = historical_data['limit_addition_amount']/historical_data['old_limit']
historical_data['limit_usage_changes'] = historical_data['limit_usage_new']-historical_data['limit_usage_old']
historical_data['limit_usage_rate_new'] = historical_data['limit_usage_new']/historical_data['new_limit']
historical_data['limit_usage_rate_old'] = historical_data['limit_usage_old']/historical_data['old_limit']
historical_data['limit_usage_rate_changes'] = (historical_data['limit_usage_rate_new']-historical_data['limit_usage_rate_old'])/historical_data['limit_usage_rate_old']

C:\Users\Cermati\Anaconda3\lib\site-packages\google\auth\_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
# print(historical_data.head())
print(historical_data.dtypes)
print(historical_data.isna().any())

cla_group                    object
partner                      object
comparison_notes             object
orderId                      object
upgrade_limit_date           object
paylater_status              object
old_limit                   float64
new_limit                   float64
limit_usage_new             float64
limit_usage_old             float64
first_trx_after_upgrade      object
limit_addition_amount       float64
limit_addition_rate         float64
limit_usage_changes         float64
limit_usage_rate_new        float64
limit_usage_rate_old        float64
limit_usage_rate_changes    float64
dtype: object
cla_group                   False
partner                     False
comparison_notes            False
orderId                     False
upgrade_limit_date          False
paylater_status             False
old_limit                   False
new_limit                   False
limit_usage_new             False
limit_usage_old             False
first_trx_after_upgrade     False


In [5]:
def set_category(df) : 
    category_by_usage_rate = ''
    if (df['limit_usage_rate_new'] == 0) & (df['limit_usage_rate_old'] == 0) : 
        category_by_usage_rate = '0. Not Used at all'
    elif (df['limit_usage_rate_new'] == df['limit_usage_rate_old']) :
        category_by_usage_rate = '1. No Changes'
    elif (df['limit_usage_rate_new']>df['limit_usage_rate_old']) & (df['limit_usage_rate_old']==0) :
        category_by_usage_rate = '1.a. Usage Increased (No Previous Usage)'
    elif (df['limit_usage_rate_new']>df['limit_usage_rate_old']) & (df['limit_usage_rate_old']>0) :
        category_by_usage_rate = '1.b. Usage Increased (Has Previous Usage)'
    elif (df['limit_usage_rate_new']<df['limit_usage_rate_old']) & (df['limit_usage_rate_new']>0) :
        category_by_usage_rate = '2.a. Usage Decreased (Has Current Usage)'
    elif (df['limit_usage_rate_new']<df['limit_usage_rate_old']) & (df['limit_usage_rate_new']==0) :
        category_by_usage_rate = '2.b. Usage Decreased (No Current Usage)'
    df['category_by_usage_rate']=category_by_usage_rate

    category_by_usage_changes = ''
    if (df['limit_usage_new'] == 0) & (df['limit_usage_old'] == 0) : 
        category_by_usage_changes = '0. Not Used at all'
    elif (df['limit_usage_new'] == df['limit_usage_old']) :
        category_by_usage_changes = '1. No Changes'
    elif (df['limit_usage_new']>df['limit_usage_old']) & (df['limit_usage_old']==0) :
        category_by_usage_changes = '1.a. Usage Increased (No Previous Usage)'
    elif (df['limit_usage_new']>df['limit_usage_old']) & (df['limit_usage_old']>0) :
        category_by_usage_changes = '1.b. Usage Increased (Has Previous Usage)'
    elif (df['limit_usage_new']<df['limit_usage_old']) & (df['limit_usage_new']>0) :
        category_by_usage_changes = '2.a. Usage Decreased (Has Current Usage)'
    elif (df['limit_usage_new']<df['limit_usage_old']) & (df['limit_usage_new']==0) :
        category_by_usage_changes = '2.b. Usage Decreased (No Current Usage)'
    df['category_by_usage_changes']=category_by_usage_changes

    category_usage_to_old_limit = ''
    if df['limit_usage_new']>df['old_limit'] : 
        category_usage_to_old_limit = 'Current Usage exceed old limit'
    else :
        category_usage_to_old_limit = ''
    df['category_usage_to_old_limit']=category_usage_to_old_limit

    category_first_trx_after_upgrade = ''
    if df['first_trx_after_upgrade'] == 'Yes' : 
        category_first_trx_after_upgrade = '1st Transact After Upgrade'
    elif df['first_trx_after_upgrade'] == 'Never Trx' : 
        category_first_trx_after_upgrade = 'Never Trx'
    else :
        category_first_trx_after_upgrade = ''
    df['category_first_trx_after_upgrade']=category_first_trx_after_upgrade

    return df

In [6]:
historical_data = historical_data.apply(set_category, axis=1)

In [7]:
historical_data.to_excel('Data_Limit_Utilization_ToCurrentDate_20210910.xlsx',index=False)

In [ ]:
pd.set_option('display.max_colwidth', None)
print(historical_data.groupby(['cla_group','comparison_notes','partner','category_by_usage_changes'],dropna=False).agg({
    'orderId' : 'count'
}))